# ISLR Chapter 5 - End of Chapter Applied exercises code portion solutions¶

Exercises from: "An Introduction to Statistical Learning with Applications in R" (Springer, 2013) by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani

Data: http://faculty.marshall.usc.edu/gareth-james/ISL/data.html

Code Solutions in Python by Arthur Avila

### 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the `Default` data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

#### (a) Fit a logistic regression model that uses income and balance to predict default .

In [1]:
import numpy as np
import pandas as pd
Default = pd.read_csv('Default.csv')
Default.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
Unnamed: 0    10000 non-null int64
default       10000 non-null object
student       10000 non-null object
balance       10000 non-null float64
income        10000 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 390.8+ KB


In [2]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

y = Default.default=='Yes'
features = ['balance','income']
X = np.c_[np.ones(len(y)),Default[features]]

model = smf.glm('default~balance+income', data = Default, family=sm.families.Binomial()).fit()
print(model.summary())

model2 = sm.Logit(y,X).fit()
print(model2.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9997
Model Family:                             Binomial   Df Model:                            2
Link Function:                               logit   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:                -789.48
Date:                             Thu, 03 Oct 2019   Deviance:                       1579.0
Time:                                     12:31:16   Pearson chi2:                 6.95e+03
No. Iterations:                                  9                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:  

  i. Split the sample set into a training set and a validation set.  

  ii. Fit a multiple logistic regression model using only the training observations.  
  
  iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.  
  
  iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [3]:
import random
random.seed(999)
[random.sample(range(10),5), random.choices(range(10),k=5)]

[[1, 8, 7, 3, 9], [8, 6, 6, 6, 8]]

In [4]:
def tt_split(y,X,seed=1478523690):
    random.seed(seed)
    train = np.sort(random.sample(range(len(y)),round(3*len(y)/4)))
    test = [x for x in range(10000) if x not in train]
    return(y[train], y[test], X[train,:], X[test,:])

def reportf(seed=1478523690):
    y_train, y_test, X_train, X_test = tt_split(y,X,seed)

    model1 = sm.Logit(y_train,X_train).fit()
    predicted = model1.predict(X_test)>.5
    confusion = pd.crosstab(predicted, y_test)
    print('\n',confusion)

    error = np.mean(predicted!=y_test)
    print('\nError rate:',error)

reportf()

Optimization terminated successfully.
         Current function value: 0.081664
         Iterations 10

 default  False  True 
row_0                
False     2414     58
True         6     22

Error rate: 0.0256


### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [5]:
[reportf(seed) for seed in (2001, 36, 111111)]

Optimization terminated successfully.
         Current function value: 0.077294
         Iterations 10

 default  False  True 
row_0                
False     2405     55
True         8     32

Error rate: 0.0252
Optimization terminated successfully.
         Current function value: 0.078460
         Iterations 10

 default  False  True 
row_0                
False     2406     53
True        14     27

Error rate: 0.0268
Optimization terminated successfully.
         Current function value: 0.078858
         Iterations 10

 default  False  True 
row_0                
False     2405     54
True        15     26

Error rate: 0.0276


[None, None, None]

### (d) Now consider a logistic regression model that predicts the probability of default using `income`, `balance`, and a dummy variable for `student`. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [6]:
student = Default.student=='Yes'
X = np.c_[X,student]
reportf()

Optimization terminated successfully.
         Current function value: 0.081232
         Iterations 10

 default  False  True 
row_0                
False     2413     59
True         7     21

Error rate: 0.0264


### 6. We continue to consider the use of a logistic regression model to predict the probability of default using `income` and `balance` on the `Default` data set. In particular, we will now compute estimates for the standard errors of the `income` and `balance` logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the `glm()` function. Do not forget to set a random seed before beginning your analysis.

#### (a) Using the `summary()` and `glm()` functions, determine the estimated standard errors for the coefficients associated with `income` and `balance` in a multiple logistic regression model that uses both predictors.

In [7]:
random.seed(42)
X = X[:,0:3]
model = sm.Logit(y,X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Thu, 03 Oct 2019   Pseudo R-squ.:                  0.4594
Time:                        12:31:18   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
x1             0.0056      0

### (b) Write a function, `boot.fn()`, that takes as input the `Default` data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [8]:
def bootf(y,X):
    idbs = random.choices(range(len(y)),k=len(y))
    yb = y[idbs]
    Xb = X[idbs,:]
    model = sm.Logit(yb,Xb).fit(disp=False)
    return([x for x in model.params[1:]])

bootf(y,X)

[0.005497426814658693, 2.3440937491704654e-05]

### (c) Use the `boot()` function together with your `boot.fn()` function to estimate the standard errors of the logistic regression coefficients for `income` and `balance`.

In [9]:
def boot(y,X,bootf,R):
    mat = np.stack([bootf(y,X) for i in range(R)])
    original = sm.Logit(y,X).fit(disp=False).params[1:]
    bias = original - mat.mean(axis=0)
    std = mat.std(axis=0)
    summ = {'original': original, 'bias': bias, 'Standard_Deviation': std}
    print(pd.DataFrame(summ))
    return summ

bs = boot(y,X,bootf,1000)

    original          bias  Standard_Deviation
x1  0.005647  8.305576e-07            0.000229
x2  0.000021  7.452845e-09            0.000005


#### (d) Comment on the estimated standard errors obtained using the `glm()` function and using your bootstrap function.

### 7. In Sections 5.3.2 and 5.3.3, we saw that the `cv.glm()` function can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just the `glm()` and `predict.glm()` functions, and a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the `Weekly` data set. Recall that in the context of classification problems, the LOOCV error is given in (5.4).

#### (a) Fit a logistic regression model that predicts `Direction` using `Lag1` and `Lag2`.

In [10]:
Weekly = pd.read_csv('Weekly.csv')
print(Weekly.info())
y = Weekly.Direction == 'Up'
X = np.c_[np.ones(len(y)),Weekly[['Lag1','Lag2']]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 10 columns):
Unnamed: 0    1089 non-null int64
Year          1089 non-null int64
Lag1          1089 non-null float64
Lag2          1089 non-null float64
Lag3          1089 non-null float64
Lag4          1089 non-null float64
Lag5          1089 non-null float64
Volume        1089 non-null float64
Today         1089 non-null float64
Direction     1089 non-null object
dtypes: float64(7), int64(2), object(1)
memory usage: 85.2+ KB
None


In [11]:
model = sm.Logit(y,X).fit(disp=0)
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:              Direction   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1086
Method:                           MLE   Df Model:                            2
Date:                Thu, 03 Oct 2019   Pseudo R-squ.:                0.005335
Time:                        12:31:42   Log-Likelihood:                -744.11
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                   0.01848
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2212      0.061      3.599      0.000       0.101       0.342
x1            -0.0387      0.026     -1.477      0.140      -0.090       0.013
x2             0.0602      0.027      2.270      0.0

#### (b) Fit a logistic regression model that predicts `Direction` using `Lag1` and `Lag2` *using all but the first observation.*

In [12]:
idx = [i for i in range(len(y)) if i!=0]
model = sm.Logit(y[idx],X[idx,]).fit(disp=0)
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:              Direction   No. Observations:                 1088
Model:                          Logit   Df Residuals:                     1085
Method:                           MLE   Df Model:                            2
Date:                Thu, 03 Oct 2019   Pseudo R-squ.:                0.005387
Time:                        12:31:43   Log-Likelihood:                -743.26
converged:                       True   LL-Null:                       -747.29
Covariance Type:            nonrobust   LLR p-value:                   0.01785
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2232      0.061      3.630      0.000       0.103       0.344
x1            -0.0384      0.026     -1.466      0.143      -0.090       0.013
x2             0.0608      0.027      2.291      0.0

#### (c) Use the model from (b) to predict the direction of the first observation. You can do this by predicting that the first observation will go up if $P(\mathtt{Direction}=\text{"Up"} | \mathtt{Lag1} , \mathtt{Lag2})> 0.5$. Was this observation correctly classified?

In [13]:
print(model.predict(X[0,], linear=1))
print(model.predict(X[0,], linear=1)>.5)
print(y[0])

[0.28753405]
[False]
False


#### (d) Write a for loop from $i = 1$ to $i = n$, where $n$ is the number of observations in the data set, that performs each of the following steps:

  i. Fit a logistic regression model using all but the ith obser vation to predict `Direction` using `Lag1` and `Lag2`.  

  ii. Compute the posterior probability of the market moving up for the $i$th observation.  

  iii. Use the posterior probability for the $i$th observation in order to predict whether or not the market moves up.  

  iv. Determine whether or not an error was made in predicting the direction for the $i$th observation. If an error was made, then indicate this as a 1, and otherwise indicate it as a 0.

In [14]:
error = []
for j in range(len(y)):
    idx = [i for i in range(len(y)) if i!=j]
    model = sm.Logit(y[idx],X[idx,]).fit(disp=0)
    error.append((model.predict(X[j,], linear=1)>.5)!=y[j])

#### (e) Take the average of the $n$ numbers obtained in (d)iv in order to obtain the LOOCV estimate for the test error. Comment on the results.

In [15]:
print('Error rate:',np.mean(error))

Error rate: 0.5454545454545454


### 8. We will now perform cross-validation on a simulated data set.

#### (a) Generate a simulated data set as follows:
```
> set.seed(1)
> x = rnorm(100)
> y = x - 2*x^2 + rnorm(100)
```
#### In this data set, what is $n$ and what is $p$? Write out the model used to generate the data in equation form.

In [16]:
random.seed(1)
x = [random.normalvariate(0,1) for i in range(100)]
y = [i - 2*i**2 + random.normalvariate(0,1) for i in x]
# yes, I could have used np.random here instead.

$n = 100$,  $p = 2$, $X \sim N(0,1)$
$$
Y(X=x) = x-2x^2 + \varepsilon
$$


### (b) Create a scatterplot of $X$ against $Y$. Comment on what you find.

In [17]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
plt.scatter(x,y)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

<Figure size 600x600 with 1 Axes>

#### (c) Set a random seed, and then compute the LOOCV errors that result from fitting the following four models using least squares:

  i. $Y = \beta_0 + \beta_1 X + \varepsilon$

  ii. $Y = \beta_0 + \beta_1 X + \beta_2X^2 + \varepsilon$  

  iii. $Y = \beta_0 + \beta_1 X + \beta_2X^2 + \beta_3X^3 + \varepsilon$  
  
  iv. $Y = \beta_0 + \beta_1 X + \beta_2X^2 + \beta_3X^3 + \beta_4X^4 + \varepsilon$.  

#### Note you may find it helpful to use the `data.frame()` function to create a single data set containing both `X` and `Y`.

In [18]:
x = np.asarray(x)
y = np.asarray(y)
X = [0,0,0,0]
X[0] = np.c_[np.ones(len(x)),x]
for i in range(1,4):
    X[i] = np.c_[X[i-1],x**(i+1)]

def loopf(i):
    idx = [j for j in np.arange(len(y)) if j!=i]
    model = sm.OLS(y[idx],X[n][idx,]).fit()
    pred = model.predict(X[n][i,])
    SE = (y[i]-pred)**2
    return(SE)
CV = [0,0,0,0]
for n in range(4):
    CV[n] = np.mean([loopf(i) for i in range(len(y))])
print(CV)

[8.143931587952995, 1.0645892557074124, 1.0752292426753154, 1.0473706757609613]


In [19]:
def myCV(n):
    model = sm.OLS(y,X[n]).fit()
    hatd = model.get_influence().hat_matrix_diag
    CV = np.mean(((y-model.fittedvalues)/(1-hatd))**2)
    return(CV)

CV = [myCV(n) for n in range(4)]
print(CV)
    

[8.143931587952993, 1.0645892557074126, 1.0752292426753154, 1.0473706757609609]


#### (d) Repeat (c) using another random seed, and report your results. Are your results the same as what you got in (c)? Why?

In [20]:
# short vesion using Hat matrix diagonal
print('Seed 1 sample and CV:    ', CV)
random.seed(2)
CV = [myCV(n) for n in range(4)]
print('\nSeed 1 sample, Seed 2 CV:', CV)

random.seed(2)
x = np.array([random.normalvariate(0,1) for i in range(100)])
y = np.array([i - 2*i**2 + random.normalvariate(0,1) for i in x])
X = [0,0,0,0]
X[0] = np.c_[np.ones(len(x)),x]
for i in range(1,4):
    X[i] = np.c_[X[i-1],x**(i+1)]
CV = [myCV(n) for n in range(4)]
print('\nSeed 2 sample and CV:    ', CV)

random.seed(1)
x = np.array([random.normalvariate(0,1) for i in range(100)])
y = np.array([i - 2*i**2 + random.normalvariate(0,1) for i in x])
X = [0,0,0,0]
X[0] = np.c_[np.ones(len(x)),x]
for i in range(1,4):
    X[i] = np.c_[X[i-1],x**(i+1)]
CV = [myCV(n) for n in range(4)]

Seed 1 sample and CV:     [8.143931587952993, 1.0645892557074126, 1.0752292426753154, 1.0473706757609609]

Seed 1 sample, Seed 2 CV: [8.143931587952993, 1.0645892557074126, 1.0752292426753154, 1.0473706757609609]

Seed 2 sample and CV:     [6.515454316933724, 0.9232650861777185, 0.9405719639944844, 0.9574413170015691]


#### (e) Which of the models in (c) had the smallest LOOCV error? Is this what you expected? Explain your answer.

In [21]:
print([i+1 for i in range(4) if CV[i]==np.min(CV)])

print('Expected is 2, which is the true data generation process.')
# Seed in Python did not help as much as in R :p

[4]
Expected is 2, which is the true data generation process.


#### (f) Comment on the statistical significance of the coefficient estimates that results from fitting each of the models in (c) using least squares. Do these results agree with the conclusions drawn based on the cross-validation results?

*R results differ because it has a different random generating process. For this sample, CV and statistical significance of coefficients do not match, but the statistical significance selects the correct specification with estimated confidence intervals for coefficients containing the true values of $\beta_1=1$ and $\beta_2=-2$.

In [22]:
for n in range(4):
    print(sm.OLS(y,X[n]).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     56.16
Date:                Thu, 03 Oct 2019   Prob (F-statistic):           3.00e-11
Time:                        12:31:49   Log-Likelihood:                -243.33
No. Observations:                 100   AIC:                             490.7
Df Residuals:                      98   BIC:                             495.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2105      0.280     -7.883      0.0

### 9. We will now consider the Boston housing data set, from the `MASS` library.

#### (a) Based on this data set, provide an estimate for the population mean of `medv` . Call this estimate $\hat\mu$.

In [23]:
# In Python, the Boston dataset is part of sklearn.datasets
from sklearn.datasets import load_boston
BostonData = load_boston()
#Boston = pd.DataFrame(BostonData.data, columns = BostonData.feature_names)
#Boston['MEDV'] = BostonData.target
medv = BostonData.target
print(medv.mean())

22.532806324110677


#### (b) Provide an estimate of the standard error of $\hat\mu$. Interpret this result. Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.

In [24]:
print(medv.std()/np.sqrt(len(medv)-1)) # Python uses a different denominator for std than R

0.408861147497535


#### (c) Now estimate the standard error of $\hat\mu$ using the bootstrap. How does this compare to your answer from (b)?

In [25]:
def boot(x,bootf,R):
    mat = np.stack([bootf(random.choices(x, k = len(x))) for i in range(R)])
    original = bootf(x)
    bias = original - mat.mean(axis=0)
    std = mat.std(axis=0)
    summ = {'original': original, 'bias': bias, 'Standard_Deviation': std}
    print(summ)
    return summ

bs = boot(medv,np.mean,1000)

{'original': 22.532806324110677, 'bias': 0.0018079051383423916, 'Standard_Deviation': 0.4042814738955241}


#### (d) Based on your bootstrap estimate from (c), provide a 95 % confidence interval for the mean of `medv`. Compare it to the results obtained using `t.test(Boston$medv)`. Hint: You can approximate a 95% confidence interval using the formula $\left[\hat\mu − 2SE(\hat\mu), \hat\mu + 2SE(\hat\mu)\right]$.

In [26]:
from scipy import stats
stats.t.interval(0.05, len(medv), np.mean(medv), np.std(medv))


(21.956369874068955, 23.1092427741524)

In [27]:
bs['original']+np.array([-1,1])*bs['Standard_Deviation']

array([22.12852485, 22.9370878 ])

#### (e) Based on this data set, provide an estimate, $\hat\mu_{med}$, for the median value of `medv` in the population.

In [28]:
np.median(medv)

21.2

#### (f) We now would like to estimate the standard error of $\hat\mu_{med}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.  

In [29]:
bs = boot(medv,np.median,1000)

{'original': 21.2, 'bias': 0.008300000000001972, 'Standard_Deviation': 0.3851637443997031}


#### (g) Based on this data set, provide an estimate for the tenth percentile of `medv` in Boston suburbs. Call this quantity $\hat\mu_{0.1}$ . (You can use the `quantile()` function.)

In [30]:
np.quantile(medv, 0.10)

12.75

#### (h) Use the bootstrap to estimate the standard error of $\hat\mu_{0.1}$. Comment on your findings.

In [31]:
bs = boot(medv, lambda x: np.quantile(x,0.1),1000)

{'original': 12.75, 'bias': -0.02254999999999896, 'Standard_Deviation': 0.4806027439580427}
